# M3 month

In [1]:
import numpy as np
import pyramid
import pandas as pd
import math
from scipy.stats import norm
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime
import requests
from io import BytesIO
from sklearn import metrics

## 1 TS without exogenous

In [94]:
df = pd.read_excel('M3C.xls',sheet_name='M3Month')
df = df[df.Category == 'MICRO       ']
df_results_Theta = pd.read_excel('M3Forecast.xls',sheet_name='THETA',header=None)
df_results_FPro = pd.read_excel('M3Forecast.xls',sheet_name='ForecastPro',header=None)
df_results_FX = pd.read_excel('M3Forecast.xls',sheet_name='ForcX',header=None)
df.head()

Series   N  NF      Category  Starting Year  Starting Month       1       2  \
0  N1402  68  18  MICRO                  1990               1  2640.0  2640.0   
1  N1403  68  18  MICRO                  1990               1  1680.0  1920.0   
2  N1404  68  18  MICRO                  1990               1  1140.0   720.0   
3  N1405  68  18  MICRO                  1990               1   180.0   940.0   
4  N1406  68  18  MICRO                  1990               1  2000.0  1550.0   

        3       4 ...   135  136  137  138  139  140  141  142  143  144  
0  2160.0  4200.0 ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
1   120.0  1080.0 ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
2  4860.0  1200.0 ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
3  2040.0   800.0 ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  
4  4450.0  3050.0 ...   NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  NaN  

[5 rows x 150 columns]

In [95]:
N_series = 9
N = df['N'][N_series]
NF = df['NF'][N_series]
start_year = df['Starting Year'][N_series]
start_month = df['Starting Month'][N_series]
Id = df['Series'][N_series]

K1 = N-12
K2 = N+6
train_data = np.array(df.iloc[N_series,6:K1]).astype(np.float64)
validate_data = np.array(df.iloc[N_series,K1:K2]).astype(np.float64)

In [96]:
forecast_Theta = np.array(df_results_Theta[df_results_Theta[0]==Id]).reshape(-1)[2:].astype(np.int)
forecast_FPro = np.array(df_results_FPro[df_results_FPro[0]==Id]).reshape(-1)[2:].astype(np.int)
forecast_FX = np.array(df_results_FX[df_results_FX[0]==Id]).reshape(-1)[2:].astype(np.int)
del df_results_FPro
del df_results_FX
del df_results_Theta

#### Training (Stepwise)

In [103]:
from pyramid.arima import auto_arima

stepwise_fit = auto_arima(train_data, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=True)  # set to stepwise

stepwise_fit.summary()

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=671.926, BIC=675.148, Fit time=0.014 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=665.340, BIC=671.784, Fit time=0.232 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=673.885, BIC=678.717, Fit time=0.026 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 0, 12); AIC=658.350, BIC=666.405, Fit time=0.496 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(2, 1, 1, 12); AIC=658.662, BIC=668.327, Fit time=1.065 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(2, 1, 0, 12); AIC=656.886, BIC=663.330, Fit time=0.429 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(2, 1, 0, 12); AIC=654.264, BIC=662.318, Fit time=0.944 seconds
Fit ARIMA: order=(1, 1, 2) seasonal_order=(2, 1, 0, 12); AIC=nan, 

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                   50
Model:             SARIMAX(2, 1, 1)x(2, 1, 0, 12)   Log Likelihood                -318.147
Date:                            Thu, 05 Jul 2018   AIC                            650.294
Time:                                    16:07:16   BIC                            661.570
Sample:                                         0   HQIC                           654.269
                                             - 50                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept    -66.4504     68.987     -0.963      0.335    -201.663      68.762
ar.L1          0.5006      0.191      2.626      0.009       0.127       0.874
ar.L2         -0.3463      0.197     -1.762      0.078      -0.732       0.039
ma.L1         -0.9061      0.235     -3.863      0.000      -1.366      -0.446
ar.S.L12      -0.6531      0.163     -4.002      0.000      -0.973      -0.333
ar.S.L24      -0.4897      0.212     -2.311      0.021      -0.905      -0.074
sigma2      1.468e+06   6.37e+05      2.305      0.021     2.2e+05    2.72e+06
===================================================================================
Ljung-Box (Q):                       20.82   Jarque-Bera (JB):                 0.15
Prob(Q):                              0.98   Prob(JB):                         0.93
Heteroskedasticity (H):               1.05   Skew:                             0.15
Prob(H) (two-sided):                  0.94   Kurtosis:                         3.00
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [107]:
from bokeh.plotting import figure, show, output_notebook
import pandas as pd
from bokeh.models import Span
# init bokeh
output_notebook()

def plot_arima(truth, forecasts, title="ARIMA", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    # always plot truth the same
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    
    # if no defined forecast start, start at the end
    if forecast_start is None:
        idx = np.arange(n_truth, n_truth + n_forecasts)
    else:
        idx = np.arange(forecast_start, n_forecasts)
    forecasts = pd.Series(forecasts, index=idx)
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted')
    p.line(truth.index, truth.values, color=c1, legend='Observed')
    vline = Span(location=49,dimension='height', line_color='red',line_width=1)
    p.renderers.extend([vline])
    
    return p

Loading BokehJS ...

In [108]:
in_sample_preds = stepwise_fit.predict_in_sample()
print (metrics.mean_absolute_error(train_data, in_sample_preds))

1228.7696529360778


In [109]:
show(plot_arima(train_data, in_sample_preds, 
                title="Original Series & In-sample Predictions", 
                c2='#FF0000', forecast_start=0))

In [142]:
def plot_arima_comparison(truth,truth_forecast, forecasts,forecast_1,forecast_2,forecast_3, title="ARIMA", xaxis_label='Time',
               yaxis_label='Value', c1='#A6CEE3', c2='#B2DF8A', 
               forecast_start=None, **kwargs):
    
    # make truth and forecasts into pandas series
    n_truth = truth.shape[0]
    n_forecasts = forecasts.shape[0]
    
    # always plot truth the same
    truth = pd.Series(truth, index=np.arange(truth.shape[0]))
    
    # if no defined forecast start, start at the end
    if forecast_start is None:
        idx = np.arange(n_truth, n_truth + n_forecasts)
    else:
        idx = np.arange(forecast_start, n_forecasts)
    forecasts = pd.Series(forecasts, index=idx)
    true_forecasts = pd.Series(truth_forecast, index=idx)
    forecasts_1 = pd.Series(forecast_1, index=idx)
    forecasts_2 = pd.Series(forecast_2, index=idx)
    forecasts_3 = pd.Series(forecast_3, index=idx)
    
    # set up the plot
    p = figure(title=title, plot_height=400, **kwargs)
    p.grid.grid_line_alpha=0.3
    p.xaxis.axis_label = xaxis_label
    p.yaxis.axis_label = yaxis_label
    
    # add the lines
    p.line(forecasts.index, forecasts.values, color=c2, legend='Forecasted_my', line_width = 2)
    #p.line(truth.index, truth.values, color=c1, legend='Observed')
    p.line(true_forecasts.index, true_forecasts.values, color=c1,legend='Observed',line_width=3)
    p.line(forecasts_1.index, forecasts_1.values, color='brown', legend='Forecasted_Theta')
    p.line(forecasts_2.index, forecasts_2.values, color='yellow', legend='Forecasted_FPRO')
    p.line(forecasts_3.index, forecasts_3.values, color='purple', legend='Forecasted_FX')

    #vline = Span(location=50,dimension='height', line_color='red',line_width=1)
    #p.renderers.extend([vline])
    p.legend.location = "bottom_left"
    p.legend.orientation = "horizontal"
    return p

In [143]:
show(plot_arima_comparison(train_data,validate_data,next_validate,forecast_Theta,forecast_FPro,forecast_FX))

In [72]:
print ('MAE (Stepwise):', metrics.mean_absolute_error(validate_data,next_validate))

MAE (Stepwise): 1239.4830897226109


In [101]:
# call the plotting func
show(plot_arima(train_data, forecast_Theta))
show(plot_arima(train_data, forecast_FPro))
show(plot_arima(train_data, forecast_FX))
show(plot_arima(train_data, next_validate))

In [73]:
full_prediction = np.concatenate([in_sample_preds,next_validate,forecast_FPro])
show(plot_arima(full_data, full_prediction, forecast_start=0))

In [74]:
full_prediction = np.concatenate([in_sample_preds,next_validate,forecast_FX])
show(plot_arima(full_data, full_prediction, forecast_start=0))

In [ ]:
rs_fit = auto_arima(train_data, start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                          start_P=0, seasonal=True, d=1, D=1, trace=True,
                          error_action='ignore',  # don't want to know if an order does not work
                          suppress_warnings=True,  # don't want convergence warnings
                          stepwise=False,n_fi)  # set to stepwise

rs_fit.summary()